In [94]:

import pandas as pd
import os
from configs.config_util import *
import json
cur_path = os.getcwd()
cur_path

'/home/student/kuzin/python_project/TextToSvg_MLCUP'

In [95]:
configs_name = ["Qwen3_0_6B.yaml", "Qwen3_8B.yaml","Qwen3_14B.yaml" ]

In [96]:
target =  "tifa_metrics.json"
for_pivot = []
for  config_name in configs_name:
    config_path = os.path.join(cur_path, 'configs',config_name)
    config = load_config(config_path)
    model_name = config.path
    dir_path_for_res = os.path.join(cur_path,"data", model_name)
    target_path = os.path.join(dir_path_for_res,target)
    with open(target_path, "r", encoding="utf-8") as file:
        data = json.load(file)
        data = data["accuracy_by_type"]
    df_model = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])
    
# Сбрасываем индекс, чтобы категории стали колонкой
    df_model.reset_index(inplace=True)
    df_model.columns = ['Category', 'Value']
    df_model['Model'] = model_name
    for_pivot.append(df_model)
combined_df = pd.concat(for_pivot, ignore_index=True)

# Создаем сводную таблицу
pivot_table = combined_df.pivot(index='Category', columns='Model', values='Value')
mean_row = pivot_table.mean(axis=0)

# Преобразуем в DataFrame и добавляем как новую строку
mean_row_df = pd.DataFrame([mean_row], index=['Mean'], columns=pivot_table.columns)

# Объединяем с оригинальной таблицей
pivot_table_with_mean = pd.concat([pivot_table, mean_row_df])

print(pivot_table_with_mean)

Model         Qwen/Qwen3-0.6B  Qwen/Qwen3-14B  Qwen/Qwen3-8B
activity             0.231848        0.223080       0.107682
animal/human         0.195944        0.195087       0.126821
attribute            0.276846        0.277140       0.172698
color                0.480207        0.473322       0.421113
counting             0.135903        0.137931       0.165314
food                 0.141603        0.140505       0.120746
location             0.152174        0.151630       0.088587
material             0.358852        0.334928       0.272727
object               0.156226        0.155462       0.136618
other                0.318408        0.283582       0.313433
shape                0.550725        0.434783       0.623188
spatial              0.377042        0.377042       0.311258
Mean                 0.281315        0.265374       0.238349


In [97]:
target =  "ascore_metrics.json"
for_pivot = []
for  config_name in configs_name:
    config_path = os.path.join(cur_path, 'configs',config_name)
    config = load_config(config_path)
    model_name = config.path
    dir_path_for_res = os.path.join(cur_path,"data", model_name)
    target_path = os.path.join(dir_path_for_res,target)
    df_model = pd.read_json(target_path)[["mean_generic_AScore", "percentage", "cancel_flag"]]
    df_model = df_model[df_model["cancel_flag"]==2].drop(columns=["cancel_flag"])
    df_model['Model'] = model_name
    for_pivot.append(df_model)
combined_df = pd.concat(for_pivot, ignore_index=True)
grouped_df = combined_df.groupby('Model').agg(
    mean_AScore=('mean_generic_AScore', 'mean'),
    avg_percentage=('percentage', 'mean')
).reset_index()
grouped_df

,Model,mean_AScore,avg_percentage
0,Qwen/Qwen3-0.6B,4.159800,98.030439
1,Qwen/Qwen3-14B,4.263120,90.393912
2,Qwen/Qwen3-8B,4.326931,83.133393


In [98]:
target =  "outputs_model.csv"
for_pivot = []
for  config_name in configs_name:
    config_path = os.path.join(cur_path, 'configs',config_name)
    config = load_config(config_path)
    model_name = config.path
    dir_path_for_res = os.path.join(cur_path,"data", model_name)
    target_path = os.path.join(dir_path_for_res,target)
    df_model = pd.read_csv(target_path)

In [99]:
inst_ten = 64

In [100]:
dir_path_for_reference =  os.path.join(cur_path,"data", "svg_eval_dataset.csv")
orig_ref = pd.read_csv(dir_path_for_reference)['svg_code'].head(inst_ten).tolist()

In [101]:
import pandas as pd
import os
from IPython.display import display, HTML

# Ваш существующий код
target = "outputs_model.csv"
model_svgs = {}
# Предполагается, что configs_name и load_config определены ранее
for config_name in configs_name:
    config_path = os.path.join(cur_path, 'configs', config_name)
    config = load_config(config_path)
    model_name = config.path
    dir_path_for_res = os.path.join(cur_path, "data", model_name)
    target_path = os.path.join(dir_path_for_res, target)
    
    df_model = pd.read_csv(target_path)
    model_svgs[model_name] = df_model['0'].head(inst_ten).tolist()

# Добавляем оригинальные изображения
model_svgs["original"] = orig_ref  # orig_ref должен быть списком из 10 SVG-строк

# Проверка количества изображений
min_required = inst_ten
for model, svgs in model_svgs.items():
    if len(svgs) < min_required:
        raise ValueError(f"Модель {model} содержит меньше {min_required} SVG-кодов")

# HTML-шаблон с улучшенным стилем
html_template = """
<table style="border-collapse: collapse; width: 100%; border: 1px solid #ddd; margin: 20px 0;">
  {rows}
</table>
"""

# Формируем строки таблицы
table_rows = []

# Получаем список моделей без "original"
models = [model for model in model_svgs.keys() if model != "original"]

for i in range(inst_ten):  # 10 наборов изображений
    # Строка с оригинальным изображением по центру (уменьшенный размер)
    original_svg = model_svgs["original"][i]
    
    # Добавляем стили для уменьшения размера SVG
    resized_svg = original_svg.replace('<svg', '<svg style="max-width: 300px; height: 300px;"')
    
    row = f"""
    <tr>
      <td colspan="{len(models)}" style="border: 1px solid #ddd; padding: 8px; text-align: center; background-color: #f9f9f9;">
        <div style="display: inline-block; max-width: 300px;">
          {resized_svg}
        </div>
        <div style="margin-top: 8px; font-size: 0.9em; color: #666;">Original</div>
      </td>
    </tr>
    """
    table_rows.append(row)
    
    # Строка с сгенерированными изображениями
    row = "<tr>"
    for model_name in models:
        svg_code = model_svgs[model_name][i]
        # Ограничиваем размер сгенерированных изображений
        resized_svg = svg_code.replace('<svg', '<svg style="max-width: 100%; height: auto;"')
        row += f"""
        <td style="border: 1px solid #ddd; padding: 4px; text-align: center;">
          <div style="max-width: 100%;">
            {resized_svg}
          </div>
          <div style="margin-top: 5px; font-size: 0.8em; color: #555;">{model_name}</div>
        </td>
        """
    row += "</tr>"
    table_rows.append(row)
# Финальный HTML
final_html = html_template.format(rows='\n'.join(table_rows))

# Добавляем стили и заголовок
full_html = f"""
<style>
    .svg-container {{
        max-width: 100%;
        margin: 0 auto;
    }}
    svg {{
        max-width: 100%;
        height: auto;
    }}
</style>

<div class="svg-container">
    <h3 style="text-align: center; color: #2c3e50;">Сравнение сгенерированных изображений с оригиналом</h3>
    {final_html}
</div>
"""

display(HTML(full_html))

In [82]:
import pandas as pd
import os
from IPython.display import display, HTML

target = "outputs_model.csv"
cur_path = os.getcwd()
model_svgs = {}

for config_name in configs_name:
    config_path = os.path.join(cur_path, 'configs', config_name)
    config = load_config(config_path)  # Предполагается, что функция load_config определена
    model_name = config.path
    dir_path_for_res = os.path.join(cur_path, "data", model_name)
    target_path = os.path.join(dir_path_for_res, target)
    
    df_model = pd.read_csv(target_path)
    # Сохраняем первые 10 SVG-кодов от каждой модели
    model_svgs[model_name] = df_model['0'].head(10).tolist()
model_svgs["original"] = orig_ref

# Проверка, что у всех моделей есть хотя бы 10 SVG-кодов
min_required = 10
for model, svgs in model_svgs.items():
    if len(svgs) < min_required:
        raise ValueError(f"Модель {model} содержит меньше {min_required} SVG-кодов")

# Создание HTML-таблицы для Jupyter
html_template = """
<table style="border-collapse: collapse; width: 100%; border: 1px solid #ddd;">
  <tr>
    {headers}
  </tr>
  {rows}
</table>
"""

headers = ''.join([f"<th style='border: 1px solid #ddd; padding: 8px; background-color: #f2f2f2;'>{model}</th>" 
                  for model in model_svgs.keys()])

# Создание строк таблицы
table_rows = []
for i in range(10):  # 10 строк
    row = "<tr>"
    for model_name in model_svgs.keys():
        svg_code = model_svgs[model_name][i]
        row += f"<td style='border: 1px solid #ddd; padding: 4px; text-align: center;'>{svg_code}</td>"
    row += "</tr>"
    table_rows.append(row)

final_html = html_template.format(headers=headers, rows='\n'.join(table_rows))
display(HTML(final_html))

ModuleNotFoundError: No module named 'svglib'